In [1]:
from scrapy import Selector

import requests

import pandas as pd

from tqdm.notebook import tqdm, trange
tqdm.pandas()

/Users/nicolas/me204-2024-project-ntelloyoung/me204-2024-project-ntelloyoung/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Loads dataframe which contains all games
all_games = pd.read_csv('../data/raw data/all_games.csv')

In [3]:
# Remove the ', [, and ] characters from the 'link' column in 'all_games.csv' so that scrapy can access the links
all_games['link'] = all_games['link'].str.replace('[', '')
all_games['link'] = all_games['link'].str.replace(']', '')
all_games['link'] = all_games['link'].str.replace("'", '')

In [4]:
# Function to scrape the release information of a game
def get_release_info(link):

    response = requests.get(link)

    sel = Selector(text = response.text)

    containers = sel.css('div.info-release')

    title = Selector(text = response.text).css('div.mb h1::text').get() 
    
    credits = containers.css('dd:nth-child(4) a::text').get()

    release_date = containers.css('a#txbox::text').get()

    publishers = containers.css('ul#publisherLinks a::text').getall()

    developers = containers.css('ul#developerLinks a::text').getall()

    return {
        'title': title,
        'credits': credits,
        'xbox_release_date': release_date,
        'publishers': publishers,
        'developers': developers,
    }

In [5]:
# Applies the function to all the games in the dataframe
release_info_dict = all_games['link'].progress_apply(get_release_info)

  0%|          | 0/1017 [00:00<?, ?it/s]

In [6]:
# Turn the dict into a dataframe
release_info = pd.DataFrame(release_info_dict.tolist())

In [7]:
# Save the dataframe
release_info.to_csv("../data/raw data/release_info.csv", index=False)